Name: Avani Kasat

Contact no: 8148520593


In [1]:
#import all necessary libraries
import requests
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
from urllib.request import urlopen
import os
from zipfile import ZipFile

Acquire data:

1. Programatically fetch “Securities available for Equity segment (.csv)” file From the URL: https://www.nseindia.com/market-data/securities-available-for-trading

In [2]:
url= 'https://archives.nseindia.com/content/equities/EQUITY_L.csv'
df = pd.read_csv(url)
df.to_csv('Equity_Segment.csv')

In [3]:
df.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,EQ,03-MAY-1995,10,1,INE253B01015,10
2,3IINFOLTD,3i Infotech Limited,EQ,22-OCT-2021,10,1,INE748C01038,10
3,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10
4,3PLAND,3P Land Holdings Limited,EQ,19-JUL-1995,2,1,INE105C01023,2


In [4]:
df.describe

<bound method NDFrame.describe of           SYMBOL                           NAME OF COMPANY  SERIES  \
0      20MICRONS                        20 Microns Limited      EQ   
1     21STCENMGM  21st Century Management Services Limited      EQ   
2      3IINFOLTD                       3i Infotech Limited      EQ   
3        3MINDIA                          3M India Limited      EQ   
4         3PLAND                  3P Land Holdings Limited      EQ   
...          ...                                       ...     ...   
1848        ZOTA                  Zota Health Care LImited      EQ   
1849       ZUARI              Zuari Agro Chemicals Limited      EQ   
1850    ZUARIIND                  ZUARI INDUSTRIES LIMITED      EQ   
1851   ZYDUSLIFE                Zydus Lifesciences Limited      EQ   
1852   ZYDUSWELL                    Zydus Wellness Limited      EQ   

      DATE OF LISTING   PAID UP VALUE   MARKET LOT   ISIN NUMBER   FACE VALUE  
0         06-OCT-2008               5        

2. Programatically get the latest “bhavcopy” csv file from the following URL - https://www.nseindia.com/all-reports

In [5]:
today = datetime.datetime.now()
today = today -timedelta(days=1)
yr = str(today.year)
m = today.month
# Helper Function to get month
def get_month(m):
    if(m==1):
        month = "JAN"
    if(m==2):
        month = "FEB"
    if(m==3):
        month = "MAR"
    if(m==4):
        month = "APR"
    if(m==5):
        month = "MAY"
    if(m==6):
        month = "JUN"
    if(m==7):
        month = "JUL"
    if(m==8):
        month = "AUG"
    if(m==9):
        month = "SEP"
    if(m==10):
        month = "OCT"
    if(m==11):
        month = "NOV"
    if(m==12):
        month = "DEC"
    return month
month=get_month(m)
d = '{:02d}'.format(today.day)
d = str(d)
current_date = d+month+yr
url2 = 'https://archives.nseindia.com/content/historical/EQUITIES/2022/'+f"{month}"+'/cm'+f"{current_date}"+'bhav.csv.zip'
r = requests.head(url2)
if r.status_code == 503:

    df = pd.read_csv(url2)
    df.to_csv('bhavcopies.csv')

3. Construct a (relational) database with normalized tables & insert both the data files into it

In [7]:
# load the data into a Pandas DataFrame
merged = pd.read_csv('bhavcopies.csv').merge(pd.read_csv('Equity_Segment.csv'),on=["SYMBOL"])
merged.drop("Unnamed: 0_x",inplace=True,axis=1)
merged.drop("Unnamed: 0_y",inplace=True,axis=1)
merged.drop("Unnamed: 13",inplace=True,axis=1)
merged.to_csv('Data_file_merged.csv')

4. In addition to step 2, programmatically get bhavcopies of the last 30 days instead of just the latest one.

In [20]:
today1 = datetime.datetime.now()
today1 = today1 - timedelta(days=1) 
count = 1
i = 0
while 1:
    yesterday = today1 - timedelta(days=i)
    dt = today - timedelta(days=i)
    i+=1
    
    x = dt.weekday()
    # print(x)
    yr = str(yesterday.year)
    m = yesterday.month
    month=get_month(m)
    d = '{:02d}'.format(yesterday.day)
    d = str(d)
    if x==5 or x==6:   
        continue
    if d=='00':
        continue
    current_date = d+month+yr
    if current_date == '08NOV2022' :
        continue
    if current_date == '26OCT2022' :
        continue 
    print(current_date)
    url2 = 'https://archives.nseindia.com/content/historical/EQUITIES/2022/'+f"{month}"+'/cm'+f"{current_date}"+'bhav.csv.zip'
    r = requests.get(url2)
    
    if r.status_code == 200:
        
        df = pd.read_csv(url2)
        count+=1
        df.to_csv('bhavcopies'+f"{count}"+'.csv')
        
    if count ==30:
        break 

06DEC2022
05DEC2022
02DEC2022
01DEC2022
30NOV2022
29NOV2022
28NOV2022
25NOV2022
24NOV2022
23NOV2022
22NOV2022
21NOV2022
18NOV2022
17NOV2022
16NOV2022
15NOV2022
14NOV2022
11NOV2022
10NOV2022
09NOV2022
07NOV2022
04NOV2022
03NOV2022
02NOV2022
01NOV2022
31OCT2022
28OCT2022
27OCT2022
25OCT2022


Queries:

1. Write a SQL query to fetch the top 25 gainers of the day sorted in the order of their gains. Gains is defined as [(close - open) / open] for the day concerned as per point 2 above.


In [21]:
import pandas 
import sqlite3
import time
df = pandas.read_csv("Data_file_merged.csv")
table_name = "MY_table"
column = df["TIMESTAMP"]
print(column[0])
f = open("real.db", "w")
conn = sqlite3.connect('real.db')
df.to_sql(table_name, conn, if_exists='append', index=False)

crsr = conn.cursor()
# SQL command to create a table in the database
# execute the statement
crsr.execute('''SELECT `NAME OF COMPANY`, ((CLOSE-OPEN)/(OPEN)) AS Gain FROM MY_table ORDER BY (Gain) DESC limit 25''')
data = crsr.fetchall()
print("TOP 25 QUERIES ARE AS FOLLOWS :")
for d in data:
    print(d,end="\n")
# close the connection
conn.close()



06-DEC-2022
TOP 25 QUERIES ARE AS FOLLOWS :
('Univastu India Limited', 0.21217948717948715)
('Five-Star Business Finance Limited', 0.20777337329594905)
('Punjab & Sind Bank', 0.17519685039370092)
('Dhunseri Tea & Industries Limited', 0.17186463171864616)
('Tijaria Polypipes Limited', 0.16161616161616157)
('Sumeet Industries Limited', 0.13761467889908258)
('Jagsonpal Pharmaceuticals Limited', 0.10861027190332323)
('Shriram Transport Finance Company Limited', 0.10558375634517767)
('Kothari Petrochemicals Limited', 0.1020721412125862)
('Jindal Stainless Limited', 0.09977703455964329)
('PNC Infratech Limited', 0.09496051147047763)
('Ponni Sugars (Erode) Limited', 0.09470499243570352)
('Navkar Corporation Limited', 0.09406779661016944)
('Indbank Merchant Banking Services Limited', 0.09386281588447659)
('TARC Limited', 0.09070294784580499)
('Donear Industries Limited', 0.08841672378341334)
('Tracxn Technologies Limited', 0.08690012970168616)
('Visesh Infotecnics Limited', 0.08333333333333341

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


2. Get datewise top 25 gainers for last 30 days as per point 4 above.

In [22]:

count =1
for i in range(30):
    df = pandas.read_csv('bhavcopies'+f"{count}"+'.csv')
    column = df["TIMESTAMP"]
    print(column[0])
    merged = pandas.read_csv('bhavcopies'+f"{count}"+'.csv')
    merged.to_csv('bhavcopy'+f"{count}"+'.csv')
    df = pandas.read_csv('/content'+'/bhavcopy'+f"{count}"+'.csv')
    table_name = "MY_table"
    f = open('real'+f"{count}"+'.db', "w")
    conn = sqlite3.connect('real'+f"{count}"+'.db')
    df.to_sql(table_name, conn, if_exists='append', index=False)

    crsr = conn.cursor()
    crsr.execute('''SELECT SYMBOL, ((CLOSE-OPEN)/(OPEN)) AS Gain FROM MY_table ORDER BY (Gain) DESC limit 25''')
    data = crsr.fetchall()
    print("TOP 25 QUERIES ARE AS FOLLOWS :")
    for d in data:
        print(d,end="\n")
    count+=1
    print()
    time.sleep(1)
    # print(count)

06-DEC-2022
TOP 25 QUERIES ARE AS FOLLOWS :
('UNIVASTU', 0.21217948717948715)
('FIVESTAR', 0.20777337329594905)
('PSB', 0.17519685039370092)
('DTIL', 0.17186463171864616)
('JNPT', 0.16952380952380952)
('TIJARIA', 0.16161616161616157)
('VIVIANA', 0.13882352941176468)
('SUMEETINDS', 0.13761467889908258)
('PASHUPATI', 0.12275594371664246)
('JAGSNPHARM', 0.10861027190332323)
('SRTRANSFIN', 0.10558375634517767)
('KOTHARIPET', 0.1020721412125862)
('JSL', 0.09977703455964329)
('PNCINFRA', 0.09496051147047763)
('PONNIERODE', 0.09470499243570352)
('NAVKARCORP', 0.09406779661016944)
('INDBANK', 0.09386281588447659)
('TARC', 0.09070294784580499)
('DONEAR', 0.08841672378341334)
('TRACXN', 0.08690012970168616)
('VISESHINFO', 0.08333333333333341)
('ZENITHSTL', 0.08247422680412379)
('DPSCLTD', 0.08178438661710048)
('SOMICONVEY', 0.07865168539325858)
('IDBI', 0.07706422018348628)

06-DEC-2022
TOP 25 QUERIES ARE AS FOLLOWS :
('UNIVASTU', 0.21217948717948715)
('FIVESTAR', 0.20777337329594905)
('PSB', 0.